In [3]:
import pandas as pd
import plotly.express as px
from src import config
from tqdm import tqdm
import pandas as pd
from src.utils.llm.openai_client import get_proposer
predictions = pd.read_parquet("output/predictions_deepseek_chat.parquet")


predictions

,vote_id,type,title,drucksache_id,beschlussempfehlung,summary,summary_embedding,date,proposers,party,reasoning,prediction,ground_truth,category,is_governing,bundestag,vote_correct,is_own_proposal,manifesto_context
0,0,Gesetzentwurf,Gesetzentwurf der Fraktionen der SPD und CDU/C...,20/15096,None,"Abgestimmt wird über eine Grundgesetzänderung,...","[-0.00992498267441988, 0.019715214148163795, 0...",2025-03-18,"[SPD, Union]",AfD,stimmt nicht zu \nBegründung: Die Partei setz...,Ablehnung,Ablehnung,"Verteidigung & Sicherheit - Militär, Verteidig...",False,20,True,False,- Arbeitslosenversicherung reformieren: Anspru...
1,1,Änderungsantrag,Änderungsantrag der Fraktion der FDP zu der zw...,20/15120,None,Der Bundestag stimmt über die Änderung des Gru...,"[0.015315207652747631, 0.04166547954082489, 0....",2025-03-18,[FDP],AfD,stimmt nicht zu \nBegründung: Die Partei spri...,Ablehnung,Ablehnung,"Verteidigung & Sicherheit - Militär, Verteidig...",False,20,True,False,- Innere Sicherheit stärken: Bundespolizei per...
2,2,Gesetzentwurf,Gesetzentwurf der Fraktion der FDP Entwurf ein...,20/15099,None,Im Bundestag wird über die Erweiterung des bes...,"[0.014410934410989285, 0.03800387308001518, 0....",2025-03-18,[FDP],AfD,stimmt nicht zu \nBegründung: Die Partei setz...,Ablehnung,Ablehnung,"Verteidigung & Sicherheit - Militär, Verteidig...",False,20,True,False,- Innere Sicherheit stärken: Bundespolizei per...
3,3,Gesetzentwurf,Gesetzentwurf der Fraktion der CDU/CSU Entwurf...,20/12804,None,Das Gesetz stellt die Begrenzung der Zuwanderu...,"[0.03507666289806366, 0.08015237003564835, 0.0...",2025-01-31,[Union],AfD,stimmt zu \nBegründung: Der Antrag passt zum ...,Annahme,Annahme,"Inneres & Migration - Innere Sicherheit, öffen...",False,20,True,False,- Rückführungsoffensive: Ausreisepflichtige ko...
4,10,Gesetzentwurf,"Gesetzentwurf der Fraktionen SPD, CDU/CSU, BÜN...",20/12977,None,Im Bundestag wird über eine Verfassungsänderun...,"[-0.01373820099979639, 0.024543749168515205, 0...",2024-12-19,"[SPD, Union, DIE_GRÜNEN, FDP]",AfD,stimmt zu \nBegründung: Die Partei setzt sich...,Annahme,Ablehnung,"Justiz & Verbraucherschutz - Rechtsprechung, G...",False,20,False,False,- Demokratie und Rechtsstaat\n - Einführung v...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3457,677,Antrag,Antrag der Bundesregierung Fortsetzung der Bet...,17/11685,Annahme,Der Bundestag stimmt der Fortsetzung des Einsa...,"[-0.028494488447904587, 0.030802208930253983, ...",2013-01-31,[Bundesregierung],Union,stimmt zu \nBegründung: Die Partei unterstütz...,Annahme,Annahme,"Verteidigung & Sicherheit - Militär, Verteidig...",True,17,True,True,- Anpassung des Jugendstrafrechts mit Fokus au...
3458,679,Antrag,Antrag der Bundesregierung Entsendung bewaffne...,17/11783,Annahme,Der Bundestag stimmt der Entsendung bewaffnete...,"[-0.03531566634774208, 0.022582806646823883, 0...",2012-12-14,[Bundesregierung],Union,stimmt zu \nBegründung: Die Partei unterstütz...,Annahme,Annahme,"Verteidigung & Sicherheit - Militär, Verteidig...",True,17,True,True,- Anpassung des Jugendstrafrechts mit Fokus au...
3459,683,Antrag,Antrag der Bundesregierung Fortsetzung des Ein...,17/11466,Annahme,Der Bundestag stimmt der Fortsetzung des Einsa...,"[-0.010960222221910954, 0.06010444834828377, 0...",2012-12-13,[Bundesregierung],Union,stimmt zu \nBegründung: Das Wahlprogramm der ...,Annahme,Annahme,"Verteidigung & Sicherheit - Militär, Verteidig...",True,17,True,True,- Anpassung des Jugendstrafrechts mit Fokus au...
3460,684,Antrag,"Antrag der Abgeordneten Johanna Voß, Ulla Lötz...",17/11328,Ablehnung,Der Bundestag soll über einen Antrag abstimmen...,"[-0.012890162877738476, 0.056123919785022736, ...",2012-12-13,[DIE_LINKE],Union,stimmt nicht zu \nBegründung: Das Wahlprogram...,Ablehnung,Ablehnung,"Umwelt, Klima & Naturschutz - Umweltschutz, Kl...",True,17,True,False,- Stärkung der Investitionsfähigkeit des Mitte...
